In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import itertools
%matplotlib inline

In [9]:
# Load all the data of 16 seasons 
df_2020 = pd.read_csv('S2020.csv')
df_2019 = pd.read_csv('S2019.csv')
df_2018 = pd.read_csv('S2018.csv')
df_2017 = pd.read_csv('S2017.csv')
df_2016 = pd.read_csv('S2016.csv')
df_2015 = pd.read_csv('S2015.csv')
df_2014 = pd.read_csv('S2014.csv')
df_2013 = pd.read_csv('S2013.csv')
df_2012 = pd.read_csv('S2012.csv')
df_2011 = pd.read_csv('S2011.csv')
df_2010 = pd.read_csv('S2010.csv')
df_2009 = pd.read_csv('S2009.csv')
df_2008 = pd.read_csv('S2008.csv')
df_2007 = pd.read_csv('S2007.csv')
df_2006 = pd.read_csv('S2006.csv')
df_2005 = pd.read_csv('S2005.csv')

In [10]:
def parse_date(date):
    """parse the date and change it to uniform format"""
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%y').date()
    

def parse_date_other(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%Y').date()

In [11]:
# apply the function to each dataset
df_2020.Date = df_2020.Date.apply(parse_date_other)
df_2019.Date = df_2019.Date.apply(parse_date_other)
df_2018.Date = df_2018.Date.apply(parse_date_other)
df_2017.Date = df_2017.Date.apply(parse_date_other)
df_2016.Date = df_2016.Date.apply(parse_date)
df_2015.Date = df_2015.Date.apply(parse_date_other)
df_2014.Date = df_2014.Date.astype('str')
df_2014 = df_2014.iloc[:-1 , :]
df_2014.Date = df_2014.Date.apply(parse_date)
df_2013.Date = df_2013.Date.apply(parse_date)
df_2012.Date = df_2012.Date.apply(parse_date)
df_2011.Date = df_2011.Date.apply(parse_date)
df_2010.Date = df_2010.Date.apply(parse_date)
df_2009.Date = df_2009.Date.apply(parse_date)
df_2008.Date = df_2008.Date.apply(parse_date)
df_2007.Date = df_2007.Date.apply(parse_date)
df_2006.Date = df_2006.Date.apply(parse_date)
df_2005.Date = df_2005.Date.apply(parse_date)

In [12]:
# the columns necessary to make our prediction
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','B365H','B365D','B365A']

In [13]:
# apply the necessary columns
playing_statistics_20 = df_2020[columns_req]  
playing_statistics_19 = df_2019[columns_req]  
playing_statistics_18 = df_2018[columns_req]  
playing_statistics_17 = df_2017[columns_req]  
playing_statistics_16 = df_2016[columns_req]  
playing_statistics_15 = df_2015[columns_req]  
playing_statistics_14 = df_2014[columns_req]  
playing_statistics_13 = df_2013[columns_req]  
playing_statistics_12 = df_2012[columns_req]  
playing_statistics_11 = df_2011[columns_req]     
playing_statistics_10 = df_2010[columns_req]  
playing_statistics_09 = df_2009[columns_req]  
playing_statistics_08 = df_2008[columns_req]  
playing_statistics_07 = df_2007[columns_req]  
playing_statistics_06 = df_2006[columns_req]  
playing_statistics_05 = df_2005[columns_req]  

In [14]:
def get_goals_scored(playing_stat):
    """Gets the goals scored agg arranged by teams and matchweek"""
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



def get_goals_conceded(playing_stat):
    """Gets the goals conceded agg arranged by teams and matchweek"""
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat

In [ ]:
playing_statistics_20 = get_gss(playing_statistics_20)
playing_statistics_19 = get_gss(playing_statistics_19)
playing_statistics_18 = get_gss(playing_statistics_18)
playing_statistics_17 = get_gss(playing_statistics_17)
playing_statistics_16 = get_gss(playing_statistics_16)
playing_statistics_15 = get_gss(playing_statistics_15)
playing_statistics_14 = get_gss(playing_statistics_14)
playing_statistics_13 = get_gss(playing_statistics_13)
playing_statistics_12 = get_gss(playing_statistics_12)
playing_statistics_11 = get_gss(playing_statistics_11)
playing_statistics_10 = get_gss(playing_statistics_10)
playing_statistics_09 = get_gss(playing_statistics_09)
playing_statistics_08 = get_gss(playing_statistics_08)
playing_statistics_07 = get_gss(playing_statistics_07)
playing_statistics_06 = get_gss(playing_statistics_06)
playing_statistics_05 = get_gss(playing_statistics_05)

In [16]:
def get_points(result):
    """Get points by each team in every match week"""
    
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
def get_cuml_points(matchres):
    """Get the tottal points accumilated by a teams until the point of the game"""
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points


def get_matchres(playing_stat):
    """Get match result of each team"""
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [17]:
def get_agg_points(playing_stat):
    """Get the agg point of each team"""
    
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

In [ ]:
# Apply the function to the dataset
playing_statistics_20 = get_agg_points(playing_statistics_20)
playing_statistics_19 = get_agg_points(playing_statistics_19)
playing_statistics_18 = get_agg_points(playing_statistics_18)
playing_statistics_17 = get_agg_points(playing_statistics_17)
playing_statistics_16 = get_agg_points(playing_statistics_16)
playing_statistics_15 = get_agg_points(playing_statistics_15)
playing_statistics_14 = get_agg_points(playing_statistics_14)
playing_statistics_13 = get_agg_points(playing_statistics_13)
playing_statistics_12 = get_agg_points(playing_statistics_12)
playing_statistics_11 = get_agg_points(playing_statistics_11)
playing_statistics_10 = get_agg_points(playing_statistics_10)
playing_statistics_09 = get_agg_points(playing_statistics_09)
playing_statistics_08 = get_agg_points(playing_statistics_08)
playing_statistics_07 = get_agg_points(playing_statistics_07)
playing_statistics_06 = get_agg_points(playing_statistics_06)
playing_statistics_05 = get_agg_points(playing_statistics_05)

In [19]:
def get_form(playing_stat,num):
    """Get the results of last games of teams"""
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat,num):
    """Add the former results"""
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat


def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics 

In [ ]:
# Apply the changes to the data frame 
playing_statistics_20 = add_form_df(playing_statistics_20)
playing_statistics_19 = add_form_df(playing_statistics_19)
playing_statistics_18 = add_form_df(playing_statistics_18)
playing_statistics_17 = add_form_df(playing_statistics_17)
playing_statistics_16 = add_form_df(playing_statistics_16)
playing_statistics_15 = add_form_df(playing_statistics_15)
playing_statistics_14 = add_form_df(playing_statistics_14)
playing_statistics_13 = add_form_df(playing_statistics_13)
playing_statistics_12 = add_form_df(playing_statistics_12)
playing_statistics_11 = add_form_df(playing_statistics_11)
playing_statistics_10 = add_form_df(playing_statistics_10)
playing_statistics_09 = add_form_df(playing_statistics_09)
playing_statistics_08 = add_form_df(playing_statistics_08)
playing_statistics_07 = add_form_df(playing_statistics_07)
playing_statistics_06 = add_form_df(playing_statistics_06)
playing_statistics_05 = add_form_df(playing_statistics_05)

In [21]:
# Necessary columns from our new Data frame
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5','B365H','B365D','B365A' ]

In [22]:
# Apply the Changes 
playing_statistics_20 = playing_statistics_20[cols]
playing_statistics_19 = playing_statistics_19[cols]
playing_statistics_18 = playing_statistics_18[cols]
playing_statistics_17 = playing_statistics_17[cols]
playing_statistics_16 = playing_statistics_16[cols]
playing_statistics_15 = playing_statistics_15[cols]
playing_statistics_14 = playing_statistics_14[cols]
playing_statistics_13 = playing_statistics_13[cols]
playing_statistics_12 = playing_statistics_12[cols]
playing_statistics_11 = playing_statistics_11[cols]
playing_statistics_10 = playing_statistics_10[cols]
playing_statistics_09 = playing_statistics_09[cols]
playing_statistics_08 = playing_statistics_08[cols]
playing_statistics_07 = playing_statistics_07[cols]
playing_statistics_06 = playing_statistics_06[cols]
playing_statistics_05 = playing_statistics_05[cols]

# Standings Data

In [2]:
Standings = pd.read_csv('Standing(2005-2020).csv')

In [3]:
Standings.head(2)

,Unnamed: 0,Team,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Arsenal,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,2.0,5.0,6.0,8.0,18.0
1,1,Aston Villa,10.0,16.0,11.0,6.0,6.0,6.0,9.0,16.0,15.0,15.0,17.0,20.0,18.0,18.0,17.0,18.0


In [4]:
Standings = Standings.drop(columns='Unnamed: 0')

In [5]:
Standings.head(2)

,Team,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Arsenal,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,2.0,5.0,6.0,8.0,18.0
1,Aston Villa,10.0,16.0,11.0,6.0,6.0,6.0,9.0,16.0,15.0,15.0,17.0,20.0,18.0,18.0,17.0,18.0


In [6]:
Standings.shape

(45, 17)

In [7]:
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)

In [8]:
Standings.head()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Team,,,,,,,,,,,,,,,,
Arsenal,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,2.0,5.0,6.0,8.0,18.0
Aston Villa,10.0,16.0,11.0,6.0,6.0,6.0,9.0,16.0,15.0,15.0,17.0,20.0,18.0,18.0,17.0,18.0
Birmingham,12.0,18.0,18.0,19.0,18.0,9.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackburn,15.0,6.0,10.0,7.0,15.0,10.0,15.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
Blackpool,18.0,18.0,18.0,18.0,18.0,18.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0


In [35]:
def get_last(playing_stat, Standings, year):
    """Function to get the position where the team finish in the year"""
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    return playing_stat

In [36]:
# make changes
playing_statistics_20 = get_last(playing_statistics_20, Standings, 15)
playing_statistics_19 = get_last(playing_statistics_19, Standings, 14)
playing_statistics_18 = get_last(playing_statistics_18, Standings, 13)
playing_statistics_17 = get_last(playing_statistics_17, Standings, 12)
playing_statistics_16 = get_last(playing_statistics_16, Standings, 11)
playing_statistics_15 = get_last(playing_statistics_15, Standings, 10)
playing_statistics_14 = get_last(playing_statistics_14, Standings, 9)
playing_statistics_13 = get_last(playing_statistics_13, Standings, 8)
playing_statistics_12 = get_last(playing_statistics_12, Standings, 7)
playing_statistics_11 = get_last(playing_statistics_11, Standings, 6)
playing_statistics_10 = get_last(playing_statistics_10, Standings, 5)
playing_statistics_09 = get_last(playing_statistics_09, Standings, 4)
playing_statistics_08 = get_last(playing_statistics_08, Standings, 3)
playing_statistics_07 = get_last(playing_statistics_07, Standings, 2)
playing_statistics_06 = get_last(playing_statistics_06, Standings, 1)
playing_statistics_05 = get_last(playing_statistics_05, Standings, 0)

In [37]:
def get_mw(playing_stat):
    """Function to return match winner from every matches"""
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

In [38]:
#apply all the changes
playing_statistics_20 = get_mw(playing_statistics_20)
playing_statistics_19 = get_mw(playing_statistics_19)
playing_statistics_18 = get_mw(playing_statistics_18)
playing_statistics_17 = get_mw(playing_statistics_17)
playing_statistics_16 = get_mw(playing_statistics_16)
playing_statistics_15 = get_mw(playing_statistics_15)
playing_statistics_14 = get_mw(playing_statistics_14)
playing_statistics_13 = get_mw(playing_statistics_13)
playing_statistics_12 = get_mw(playing_statistics_12)
playing_statistics_11 = get_mw(playing_statistics_11)
playing_statistics_10 = get_mw(playing_statistics_10)
playing_statistics_09 = get_mw(playing_statistics_09)
playing_statistics_08 = get_mw(playing_statistics_08)
playing_statistics_07 = get_mw(playing_statistics_07)
playing_statistics_06 = get_mw(playing_statistics_06)
playing_statistics_05 = get_mw(playing_statistics_05)

In [39]:
# Concatinate all the dataframes together 
playing_stat = pd.concat([playing_statistics_20,
                          playing_statistics_19,
                          playing_statistics_18,
                          playing_statistics_17,
                          playing_statistics_16,
                          playing_statistics_15,
                          playing_statistics_14,
                          playing_statistics_13,
                          playing_statistics_12,
                          playing_statistics_11,
                          playing_statistics_10,
                          playing_statistics_09,
                          playing_statistics_08,
                          playing_statistics_07,
                          playing_statistics_06,
                          playing_statistics_05], ignore_index=True)

In [42]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

playing_stat['HTFormPtsStr'] = playing_stat['HM1'] + playing_stat['HM2'] + playing_stat['HM3'] + playing_stat['HM4'] + playing_stat['HM5']
playing_stat['ATFormPtsStr'] = playing_stat['AM1'] + playing_stat['AM2'] + playing_stat['AM3'] + playing_stat['AM4'] + playing_stat['AM5']

playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    
    """Return 1 if a team wins its last three matches"""
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    """Return 1 if a team wins its last Five matches"""
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        """Return 1 if a team lost its last three matches"""
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    
    """Return 1 if a team lost its last five matches"""
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
playing_stat['HTWinStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ws)
playing_stat['HTWinStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ws)
playing_stat['HTLossStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ls)
playing_stat['HTLossStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ls)

playing_stat['ATWinStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ws)
playing_stat['ATWinStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ws)
playing_stat['ATLossStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ls)
playing_stat['ATLossStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ls)

playing_stat.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'B365H', 'B365D', 'B365A', 'HomeTeamLP',
       'AwayTeamLP', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts',
       'ATFormPts', 'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3',
       'HTLossStreak5', 'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3',
       'ATLossStreak5'],
      dtype='object')

In [43]:
# change the data type of eatch features
playing_stat['HTGS'] = playing_stat['HTGS'].astype('int')
playing_stat['ATGS'] = playing_stat['ATGS'].astype('int')
playing_stat['HTGC'] = playing_stat['HTGC'].astype('int')
playing_stat['ATGC'] = playing_stat['ATGC'].astype('int')
playing_stat['HTP'] = playing_stat['HTP'].astype('int')
playing_stat['ATP'] = playing_stat['ATP'].astype('int')
playing_stat['HTFormPts'] = playing_stat['HTFormPts'].astype('int')
playing_stat['ATFormPts'] = playing_stat['ATFormPts'].astype('int')
playing_stat['HomeTeamLP'] = playing_stat['HomeTeamLP'].astype('float')
playing_stat['AwayTeamLP'] = playing_stat['AwayTeamLP'].astype('float')

In [44]:
# Get Goal Difference
playing_stat['HTGD'] = playing_stat['HTGS'] - playing_stat['HTGC']
playing_stat['ATGD'] = playing_stat['ATGS'] - playing_stat['ATGC']

# Diff in points
playing_stat['DiffPts'] = playing_stat['HTP'] - playing_stat['ATP']
playing_stat['DiffFormPts'] = playing_stat['HTFormPts'] - playing_stat['ATFormPts']

# Diff in last year positions
playing_stat['DiffLP'] = playing_stat['HomeTeamLP'] - playing_stat['AwayTeamLP']

In [45]:
playing_stat.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
6075,2006-05-07,Fulham,Middlesbrough,1.0,0.0,H,47,48,58,57,...,0,0,0,0,0,-11,-9,0,4,6.0
6076,2006-05-07,Man United,Charlton,4.0,0.0,H,68,41,34,51,...,0,0,0,0,0,34,-10,33,4,-17.0
6077,2006-05-07,Newcastle,Chelsea,1.0,0.0,H,46,72,42,21,...,0,1,0,0,0,4,51,-36,1,6.0
6078,2006-05-07,Portsmouth,Liverpool,1.0,3.0,A,36,54,59,24,...,0,1,1,0,0,-23,30,-41,-5,14.0
6079,2006-05-07,West Ham,Tottenham,2.0,1.0,H,50,52,54,36,...,0,0,0,0,0,-4,16,-13,-4,-1.0


In [41]:
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
playing_stat.MW = playing_stat.MW.astype(float)

for col in cols:
    playing_stat[col] = playing_stat[col] / playing_stat.MW
    # divide all results by the total game played2az

In [42]:
playing_stat.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,2020-09-12,Fulham,Arsenal,0.0,3.0,A,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,2020-09-12,Crystal Palace,Southampton,1.0,0.0,H,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2020-09-12,Liverpool,Leeds,4.0,3.0,H,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,2020-09-12,West Ham,Newcastle,0.0,2.0,A,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,2020-09-13,West Brom,Leicester,0.0,3.0,A,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [44]:
# export it to csv
playing_stat.to_csv('Epl_fea.csv')